## Start

In [2]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Source #Partner

### Links: 
___

["id",                                                                                           
 "issue_id", - это на будущее,  когда будет группировка жалоб по ишьб                                                                                    
 "src_type", какая сущность пожаловалсь (Citizen/Camera/Intercom)                                                                             
 "src_uuid", - uuid этой сущности                                                                                 
 "src_address_uuid", - адрес этой сущности                                                                      
 "src_address_type",     адрес этой сущности                                                                        
 "device_uuid", - uuid устройтсва на которое жалуется сущность                                                                              
 "device_type",                                                                
 "err_code", код ошибки intercom_offline / video / no_intercom                                                                           
 "err_src", откуда пришло MOB (мобилка) / WEB (веб интерфейсы для партнеров) / API (бэк )/ DEV (устройство)                                                                              
 "err_func",  sync (там еще будет)                                                                                   
 "aasm_state",  - статус жалобы pending / processed / failed                                                                             
 "process_error", - ошибка если статус failed                                                                           
 "actor_type", - какой живой человек с сердцем пожаловался или авто система (будет либо Citizen, Partner, auto)
 "actor_identifier", - uuid / id актера
 "created_at",
 "updated_at"]

In [7]:
# creating a table from s3

query_text = """--sql 
CREATE TABLE db1.complaints_st_partner
(
    `report_date` Date,
    `issue_id` String,
    `src_type` String,
    `src_uuid` String,
    `src_address_uuid` String,
    `src_address_type` String,
    `device_uuid` String,
    `device_type` String,
    `err_code`	String,
    `err_src` String,	
    `err_func`	String,
    `aasm_state` String,	
    `process_error`	String,
    `actor_type` String,
    `actor_identifier` String,
    `created_at` DateTime64,
    `updated_at` DateTime64,
)
ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/complaints_st_partner/year=*/month=*/*.csv', 'CSVWithNames')
PARTITION BY report_date
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [9]:
query_text = """--sql
    CREATE TABLE db1.complaints_st_partner_ch
(
    `report_date` Date,
    `issue_id` String,
    `src_type` String,
    `src_uuid` String,
    `src_address_uuid` String,
    `src_address_type` String,
    `device_uuid` String,
    `device_type` String,
    `err_code`	String,
    `err_src` String,	
    `err_func`	String,
    `aasm_state` String,	
    `process_error`	String,
    `actor_type` String,
    `actor_identifier` String,
    `created_at` DateTime64,
    `updated_at` DateTime64,
)
    ENGINE = MergeTree()
    ORDER BY src_uuid
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [10]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.complaints_st_partner_mv
    REFRESH EVERY 1 DAY OFFSET 4 HOUR RANDOMIZE FOR 1 HOUR TO db1.complaints_st_partner_ch AS
    SELECT
        *
    FROM db1.complaints_st_partner
ORDER BY report_date DESC
"""
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [11]:
query_text = """--sql
    SELECT
        *
    FROM db1.complaints_st_partner_ch
    ORDER BY report_date desc
    limit 100
    """

ch.query_run(query_text)


report_date,issue_id,src_type,src_uuid,src_address_uuid,src_address_type,device_uuid,device_type,err_code,err_src,err_func,aasm_state,process_error,actor_type,actor_identifier,created_at,updated_at
date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,datetime[ns],datetime[ns]
2025-08-18,"""""","""citizen""","""908ef78d-5fc9-49ae-b998-89e365…","""4e01aa65-a0c7-4bda-a7b7-b8f6f5…","""entry""","""a6f2de11-7101-495a-9320-8b3f38…","""camera""","""video""","""MOB""","""""","""processed""","""""","""auto""","""""",2025-08-18 12:17:51,2025-08-18 12:17:52
2025-08-18,"""""","""citizen""","""90935f37-1b78-4690-adfc-ef69c3…","""5e49c1f1-d5bf-4d4e-b9b5-05d007…","""entry""","""78444022-0ac6-4245-ac01-2b2388…","""camera""","""video""","""MOB""","""""","""processed""","""""","""auto""","""""",2025-08-18 13:30:31,2025-08-18 13:30:32
2025-08-18,"""""","""citizen""","""909b090a-684c-4739-a29b-c0252a…","""0b57004d-d862-44aa-bfdd-80f4c9…","""entry""","""5e1401f4-bd2d-4eea-aa7c-ff7559…","""camera""","""video""","""MOB""","""""","""processed""","""""","""auto""","""""",2025-08-18 12:42:29,2025-08-18 12:42:29
2025-08-18,"""""","""citizen""","""90a18ffc-b807-4d4d-97a3-d9aeca…","""49e03bf4-50e8-48b0-9b1f-88f6e6…","""entry""","""f6a4d20d-0dd8-454c-b1e7-861a1c…","""camera""","""video""","""MOB""","""""","""processed""","""""","""auto""","""""",2025-08-18 10:14:27,2025-08-18 10:14:28
2025-08-18,"""""","""citizen""","""90a77c1e-7bdd-4fa8-834e-a1c51f…","""de4f297a-5eca-454c-9726-73775a…","""entry""","""9095ba8d-35e1-4a10-99fd-36d4a8…","""camera""","""video""","""MOB""","""""","""processed""","""""","""auto""","""""",2025-08-18 13:00:51,2025-08-18 13:00:51
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-08-18,"""""","""citizen""","""92a975fd-32c7-4cba-af36-7c63e9…","""7e055132-3e80-41aa-a6c1-c8af0d…","""entry""","""9c9fa2e6-0695-4bba-ab38-b6b4c2…","""camera""","""video""","""MOB""","""""","""processed""","""""","""auto""","""""",2025-08-18 10:57:30,2025-08-18 10:57:30
2025-08-18,"""""","""citizen""","""9299ebab-3dc9-467c-97af-60deb9…","""4a782f69-95a7-47ce-b618-33c7d7…","""entry""","""d7c7b3df-cf1e-4546-9421-98b6e6…","""camera""","""video""","""MOB""","""""","""processed""","""""","""auto""","""""",2025-08-18 06:10:01,2025-08-18 06:10:02
2025-08-18,"""""","""citizen""","""92ee253b-17d6-4e91-93e0-5551da…","""1f43749a-e77c-4635-bf2b-dbff05…","""entry""","""50a77372-9bc3-4c51-9b55-7dde12…","""camera""","""video""","""MOB""","""""","""processed""","""""","""auto""","""""",2025-08-18 13:15:21,2025-08-18 13:15:22


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.complaints_st_partner_ch DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


### drop mv

In [3]:
query_text = """--sql
    DROP TABLE db1.complaints_st_partner_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop ch

In [ ]:
query_text = """--sql
    DROP TABLE db1.complaints_st_partner_ch
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.complaints_st_partner_mv
"""

ch.query_run(query_text)